<a href="https://colab.research.google.com/github/DynamicLLM/LLM2024/blob/main/src/sample-ai-agent/matrixInversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This simple example illustrates how you can integrate an LLM into a Python-based workflow to perform a mathematical task (matrix inversion) and then verify it with traditional computational methods. It also demonstrates how to prompt, parse, and validate LLM outputs, which is key to building robust AI applications.  Feel free to expand with additional logic, user interactions, or more sophisticated matrix problems.

In [52]:
!pip install openai beautifulsoup4 requests -q


In [53]:
import openai
import ast  # for safely evaluating Pythonic list representations (optional)
import numpy as np  # to make matrix multiplication easier (optional)
from openai import OpenAI

client = OpenAI(
    api_key="sk-proj-cwA"
)


In [54]:
import re
import ast

def extract_matrix_from_response(response_text):
    """
    Extract the Python list-of-lists from a code block in the LLM response.
    Returns a Python object (e.g., [[0.6, -0.7], [-0.2, 0.4]]) or None if not found.
    """
    # Regex to capture text inside triple-backticks that immediately follow "```python"
    match = re.search(r"```python\s+(.+?)\s+```", response_text, flags=re.DOTALL)
    if match:
        code_block = match.group(1)  # The text inside the ```python ... ```
        try:
            matrix = ast.literal_eval(code_block.strip())
            return matrix
        except Exception as e:
            print("Error parsing code block:", e)
            return None
    else:
        return None


In [55]:
def invert_matrix_with_llm(matrix_2x2):
    """
    Ask the LLM to invert a 2x2 matrix and parse the response.

    Args:
        matrix_2x2 (list[list[float]]): A 2x2 matrix, e.g. [[1,2],[3,4]]

    Returns:
        list[list[float]]: The 2x2 inverted matrix returned by the LLM (if parsing succeeds),
                           or None if there's an error.
    """
    # Convert matrix to a string for the LLM
    matrix_str = str(matrix_2x2)

    prompt = f"""
    You are a helpful AI. I have a 2x2 matrix: {matrix_str}.
    Please compute its inverse.
    Return the result in a Pythonic list-of-lists format, for example: [[x11, x12], [x21, x22]].
    """
    print("Prompting LLM with:", prompt)

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",   # Placeholder model name
            store=True,           # Placeholder parameter
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        response_text = completion.choices[0].message.content
        print("LLM's response:", response_text)
        # Safely parse the result as a Python list
        inverted_matrix = extract_matrix_from_response(response_text)
        return inverted_matrix
    except Exception as e:
        print("Error in LLM inversion or parsing:", e)
        return None

In [56]:
def verify_inversion(original_matrix, candidate_inverse, tolerance=1e-7):
    """
    Verify if candidate_inverse is indeed the inverse of original_matrix.

    Args:
        original_matrix (list[list[float]]): 2x2 matrix
        candidate_inverse (list[list[float]]): 2x2 matrix (candidate inverse)
        tolerance (float): numerical tolerance for identity check.

    Returns:
        bool: True if product is approximately the identity matrix, False otherwise.
    """
    # Convert to numpy arrays
    A = np.array(original_matrix, dtype=float)
    A_inv_candidate = np.array(candidate_inverse, dtype=float)

    # Matrix multiply
    product = A.dot(A_inv_candidate)

    # Identity matrix
    I = np.eye(2)

    # Check if difference from the identity is within tolerance
    return np.allclose(product, I, atol=tolerance)


In [57]:
# Define a sample 2x2 matrix
matrix_2x2 = [[4, 7],
              [2, 6]]

# 1) LLM: Invert the matrix
inverted_candidate = invert_matrix_with_llm(matrix_2x2)
print("LLM's claimed inverse:", inverted_candidate)

# 2) Verify the candidate inverse
if inverted_candidate is not None:
    is_correct = verify_inversion(matrix_2x2, inverted_candidate)
    if is_correct:
        print("✅ The LLM's inverse is correct!")
    else:
        print("❌ The LLM's inverse is NOT correct.")


Prompting LLM with: 
    You are a helpful AI. I have a 2x2 matrix: [[4, 7], [2, 6]].
    Please compute its inverse. 
    Return the result in a Pythonic list-of-lists format, for example: [[x11, x12], [x21, x22]].
    
LLM's response: To compute the inverse of a 2x2 matrix \(\begin{bmatrix} a & b \\ c & d \end{bmatrix}\), we use the formula for the inverse, which is given by:

\[
\text{Inverse} = \frac{1}{ad - bc} \begin{bmatrix} d & -b \\ -c & a \end{bmatrix}
\]

For the matrix \(\begin{bmatrix} 4 & 7 \\ 2 & 6 \end{bmatrix}\):

- \(a = 4\)
- \(b = 7\)
- \(c = 2\)
- \(d = 6\)

First, we calculate the determinant:

\[
ad - bc = (4)(6) - (7)(2) = 24 - 14 = 10
\]

Now we can compute the inverse:

\[
\text{Inverse} = \frac{1}{10} \begin{bmatrix} 6 & -7 \\ -2 & 4 \end{bmatrix} = \begin{bmatrix} 0.6 & -0.7 \\ -0.2 & 0.4 \end{bmatrix}
\]

In Pythonic list-of-lists format, the inverse of the matrix is:

```python
[[0.6, -0.7], [-0.2, 0.4]]
```
LLM's claimed inverse: [[0.6, -0.7], [-0.2, 0.4]